In [1]:
%load_ext autoreload
%autoreload

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from mirisim.config_parser import SimulatorConfig
from mirisim import MiriSimulation
import tso_lrs_sims_setup
from tso_lrs_sims_setup import wasp62_scene, wasp62_sim_config

from importlib import reload

This notebook will generate a simulated observation to support the Normal Ops data management system test run. The specifications of the programme are provided in APT proposal with PID 623 (PI: Diaz), Observation 

The dataset represented a MIRI LRS time series observation (TSO) of a generic stellar point source and:

* NGROUPS = 60
* NINTs = 200

The target for the mock observations is WASP-62, an exoplanet host star with the following properties from [the exoplanet encyclopaedia](http://exoplanet.eu/catalog/wasp-62_b/):

* spectral type F7
* T_bb = 6230 K
* K = 8.994

K magnitude of 8.994 corresponds to a flux of 20e3 microJy at 8 micron (value obtained from the ETC), and I'll use this value for the source generation. 

*NOTE*: in the APT file this target is named "FAKE SOUTH POLE".


### Steps in setting up the simulation

This notebook will go through the following steps:

* Create the scene
* Set up the simulation
* Run the simulation

Each step has its own function. Steps 1 and 2 will each write out a .ini file, which will be used as input for the final step.

In [9]:
arr = 'SLITLESSPRISM'
ngrp = 60
nints = 1


## Step 1: Creating the input scene (LRSTSO-NOPS6)

Here we'll create the input scene for the simulations using the function wasp62_scene(). Arguments:

* scene_file: the filename for the .ini file
* write_cube: write the scene image out to a FITS file (optional; default=False)

The function returns a mirisim.skysim.scenes.CompositeSkyScene object.


In [10]:
scene_ini = wasp62_scene(scene_file='lrstso_nops6_scene.ini', write_cube=False)

2019-08-08 13:46:23,773 - INFO - Initializing Point
2019-08-08 13:46:23,775 - INFO - Initializing Background


Creating scene. Scene will be written to file lrstso_nops6_scene.ini
File lrstso_nops6_scene.ini already exists. Will be overwritten.
lrstso_nops6_scene.ini exists, overwrite (y/[n])?y


In [11]:
print(scene_ini)

lrstso_nops6_scene.ini


## Step 2: Configuring the simulation

Now I'll set up the simulations and prepare to run them. I'll set it up to loop through the 6 simulations. For this I wrote the function wasp62_sim_config. Check the docstring for descriptions and default values of the arguments. 

The function will write out another .ini file containing the simulation configuration, and it returns the output filename for further use.

In [12]:
sim_ini = wasp62_sim_config(mode='lrs', arr=arr, ngrp=ngrp, nint=nints, nexp=1, scene_file=scene_ini, out=True)
print(sim_ini)
#for (a, g, i) in zip(arr, ngrp, nints):
 #   sim_ini = wasp62_sim_config(mode='lrs', arr=a, ngrp=g, nint=i, nexp=1, 
 #                  scene_file=scene_ini, out=True)
 #   print(sim_ini)

Found scene file lrstso_nops6_scene.ini
lrstso_SLITLESSPRISM_60G1I1E_simconfig.ini


### Step 3: Run the simulation

In the following step we'll run the simulations for the 6 different cases. For each run, we need 3 input files: the scene, the simulation configuration, and the simulator setup file. The first and last of these remain the same for each run, and we loop through the list of 6 simulation config files.

After the simulation has run, the code renames the output directory to include the simulation settings to the directory.


In [14]:
cfg_files = glob.glob('lrstso_*_simconfig.ini')

# configure the simulator engine - this requires no editing from the default
simulator_config = SimulatorConfig.from_default()

for f in cfg_files[:1]:
    tmp = f.split('.')
    fcomps = tmp[0].split('_')
    sim = MiriSimulation.from_configfiles(f)
    sim.run()
    outdir = sorted(glob.glob('*_*_mirisim'), key=os.path.getmtime )[-1]
    new_outdir = 'lrstso_nops6_{1}_{2}'.format(fcomps[2], outdir)
    os.rename(outdir, new_outdir)
    print(outdir, new_outdir)


2019-08-08 13:47:43,285 - INFO - Using simulation configuration: lrstso_SLITLESSPRISM_60G1I1E_simconfig.ini
2019-08-08 13:47:43,291 - INFO - Using scene configuration: lrstso_nops6_scene.ini
2019-08-08 13:47:43,297 - INFO - MIRISim version: 2.1.0
2019-08-08 13:47:43,299 - INFO - MIRI Simulation started.
2019-08-08 13:47:43,300 - INFO - Output will be saved to: 20190808_134743_mirisim
2019-08-08 13:47:43,305 - INFO - Storing configs in output directory.
2019-08-08 13:47:43,310 - INFO - Using $CDP_DIR for location of CDP files: /Users/kendrew//CDP
2019-08-08 13:47:43,311 - INFO - Setting up simulated Observation, with following settings:
2019-08-08 13:47:43,315 - INFO - Configuration Path: LRS_SLITLESS
2019-08-08 13:47:43,318 - INFO - Primary optical path: IMA
2019-08-08 13:47:43,321 - INFO - IMA Filter: P750L
2019-08-08 13:47:43,325 - INFO - IMA Subarray: SLITLESSPRISM
2019-08-08 13:47:43,330 - INFO - IMA detector readout mode: FAST
2019-08-08 13:47:43,333 - INFO - IMA detector # exposu

Exception: Invalid  cdp for lrs distortion